In [1]:
import os
import torch
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline, AutoConfig, BitsAndBytesConfig, GenerationConfig, TrainingArguments
from peft import LoraConfig, PeftModel
from datasets import Dataset, load_dataset
from trl import SFTTrainer

In [2]:
# define compute module
compute_dtype = getattr(torch, 'float16')

# Quantization parameter
bnb_config = BitsAndBytesConfig(
    load_in_4bit = True,
    bnb_4bit_quant_type = 'nf4',
    bnb_4bit_compute_dtype = compute_dtype,
    bnb_4bit_use_double_quant = True,
    
)

In [3]:
model_id = "TC14/Otto_lora_run5"

model = AutoModelForCausalLM.from_pretrained(model_id,
                                             quantization_config = bnb_config,
                                            device_map = "auto")

tokenizer = AutoTokenizer.from_pretrained(model_id)

config.json:   0%|          | 0.00/732 [00:00<?, ?B/s]

model.safetensors.index.json:   0%|          | 0.00/23.9k [00:00<?, ?B/s]

model-00001-of-00007.safetensors:   0%|          | 0.00/4.89G [00:00<?, ?B/s]

model-00002-of-00007.safetensors:   0%|          | 0.00/4.83G [00:00<?, ?B/s]

model-00003-of-00007.safetensors:   0%|          | 0.00/5.00G [00:00<?, ?B/s]

...

generation_config.json:   0%|          | 0.00/194 [00:00<?, ?B/s]

tokenizer_config.json:   0%|          | 0.00/51.2k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/9.09M [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/434 [00:00<?, ?B/s]

Special tokens have been added in the vocabulary, make sure the associated word embeddings are fine-tuned or trained.


In [7]:
messages = [{'role': 'system', 'content': 'You are an radio transcript message transcript assisant, please classifer the following message'},
             {'role': 'user', 'content': 'clear by two'}]

prompt = tokenizer.apply_chat_template(
    messages,
    return_tensors = "pt"
)

#text_input = tokenizer(prompt , return_tensors = "pt")

#print(text_input)

#model.to("cuda")
model_input = prompt.cuda()

generation_output = model.generate(
    model_input,
    max_new_tokens = 20,
    do_sample = True
)

decoded = tokenizer.batch_decode(generation_output)
print(decoded)

The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior. Please pass your input's `attention_mask` to obtain reliable results.
Setting `pad_token_id` to `eos_token_id`:128001 for open-end generation.


['<|begin_of_text|><|start_header_id|>system<|end_header_id|>\n\nYou are an radio transcript message transcript assisant, please classifer the following message<|eot_id|><|start_header_id|>user<|end_header_id|>\n\nclear by two<|eot_id|><|start_header_id|>assiant<|end_header_id|>\n\nGeneral Spotting<|eot_id|>']
